In [1]:
import subprocess
def ExecuteCmd(command_str, output_file='output_cmd/output.txt'):
  
    with open(output_file, 'w') as f:
        result = subprocess.run(
            command_str,
            shell=True,
            stderr=f, 
            stdout=f,
            text=True,
            executable='/bin/bash'
        )
    return f"Output saved to {output_file}"

In [2]:
from dotenv import load_dotenv
import os
from google import genai

load_dotenv()
api_key = os.getenv("GEMINI_API_KEY")

client = genai.Client()

In [3]:
import json

# Path to your JSON file
file_path = "Datasets/network_analysis_1/na1.json"

# Load the JSON content
with open(file_path, "r") as f:
    data = json.load(f)

# Extract parts from the JSON
scenario = data["scenario"]
tools = data["tools"]
questions = data["questions"]
file=data["files"]



In [ ]:
#custom prompt for this tool: RETURNS ONLY THE COMMAND
def Tool1(history):
    response = client.models.generate_content(
        model="gemini-2.5-pro",
        contents="\n".join([str(h) for h in history])
    )
    #history.append({"role": "model", "text": [f"Command generated by the model:\n{response.text}"]})
    
    return response.text

#not the same prompt as tool1 but take the command provided by tool1
def tool2_thought(history, thought, command):
    with open('output_cmd/output.txt', 'r') as f:
        output = f.read()

    if len(output) > 500:
        output = output[:500] + "\n...[truncated]..." #for large output file we only take the first 5000 characters 

    history.append({"previous Thought": [thought],"previous command": [command], "previous command output": [output]})

    response = client.models.generate_content(
        model="gemini-2.5-pro",
        contents="\n".join([str(h) for h in history])
    )
    return response.text

def tool2_action(history):
    response = client.models.generate_content(
        model="gemini-2.5-pro",
        contents="\n".join([str(h) for h in history])
    )
    return response.text


In [14]:
from Prompts import prompter
i = -1
nb_questions = len(questions)
history = []
final_answers= ["Answer not found"] * nb_questions  # Initialize with default answer
previous_input = None
output0 = None  # Initialize 
output1 = None
# Main control loop
while i < nb_questions:
    user_input = input("\nChoose:\n1. Next Question\n2. Analyze/Continue\n3. Exit\n>>> ")
    if previous_input is None and user_input not in ["1", "3"]:
        print("❌ You must start with option 1.")
        continue

    
    if user_input == previous_input and user_input == "1":
        print("❌ You already chose 1. You must alternate between 1 and 2.")
        continue

    previous_input = user_input  # Store the last input for validation

    if user_input == "1":
        if i == nb_questions - 1:
            print("✅ Task finished.")
            break
        i += 1
        prompt = prompter.prompt_tool1_thought(i, scenario, tools, file, questions)    #first prompt for tool1 thought
        history = [{"role": "system", "text": [prompt]}]  # start fresh for new question 
        output0 = Tool1(history)#first thought
        print(f"🧠 Thought generated for question {i+1} by tool 1: {output0}")
        prompt = prompter.prompt_tool1_action(i, scenario, tools, file, questions,output0)    #first prompt for tool1 
        history = [{"role": "system", "text": [prompt]}]  # start fresh for new question
        output1 = Tool1(history)# first command
        print(f"🛠️ Command generated for question {i+1} by tool 1: {output1}, press 2 to execute")
        ExecuteCmd(output1)  # Execute the command generated by Tool1
        #we should modify the history here for the tool2: new history becomes empty -> receives the new prompt -> receives command
        

    elif user_input == "2":
        prompt = prompter.prompt_tool2_thought(i, scenario, tools, file, questions)
        history[0]["text"]=[prompt]
        output0 = tool2_thought(history, output0, output1) #genrate tool2 thought
        print(f"🧠 Thought generated for question {i+1} by tool 2: {output0}")
        prompt = prompter.prompt_tool2_action(i, scenario, tools, file, questions, output0)
        history[0]["text"]=[prompt]
        output1 =tool2_action(history)
    
        print(f"📜 {history}")
        if "final answer:" in output1.lower():
            final_answers[i] = output1.lower().split("final answer:")[-1].strip()
            print(f"Answer to the question {i+1}: {final_answers[i]}, click 1 to move to next question")
            
        else:
            print(f"🛠️ New command: {output1}, press 2 to execute it ")
            ExecuteCmd(output1)
        
        #if final result append to list else exucute cmd
    elif user_input == "3":
        print("⛔ Stopped by user.")
        break

    else:
        print("⚠️ Invalid input. Please enter 1, 2, or 3.")


🧠 Thought generated for question 1 by tool 1: To find the private IP of the infected host, I need to analyze the network traffic to determine which internal machine is initiating suspicious outbound connections. The infected host will have an IP address from a private range (like 10.x.x.x, 172.16.x.x-172.31.x.x, or 192.168.x.x).

My approach will be to list all the network conversations (source IP to destination IP) within the PCAP file. I will then look for a single private IP address that is the source of numerous connections to various external, public IP addresses. This pattern is characteristic of an infected machine communicating with Command and Control (C2) servers. The private IP that acts as the source in these suspicious conversations is the infected host.
🛠️ Command generated for question 1 by tool 1: tshark -r Datasets/network_analysis_1/traffic-with-dridex-infection.pcap -z conv,ip, press 2 to execute
🧠 Thought generated for question 1 by tool 1: The previous command outp

ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}

In [11]:
print(final_answers)

['10.11.27.101', 'Answer not found', 'Answer not found', 'Answer not found', 'Answer not found']


In [6]:
print(prompter.prompt_tool1_thought(0, scenario, tools, file, questions))


You are solving a Blue Team Labs Online (BTLO)-style cybersecurity investigation challenge using Ubuntu terminal commands.

Scenario:
The SOC detected suspicious activity related to Dridex malware from an internal host. A PCAP was retrieved for further analysis after the infected user opened a macro-enabled document. Analyze the PCAP and answer the following questions.

Available Tools:
tshark, tcpdump

Target File(s):
Datasets/network_analysis_1/traffic-with-dridex-infection.pcap

Question:
What’s the private IP of the infected host? (format: X.X.X.X)

Instructions:
- Think step-by-step about how you would approach answering the question.
- Consider which tools or files might be relevant.
- Do not give a command yet.

Respond in this format only:
<your reasoning about what to do next>

Important:
- Do not provide any commands.
- Focus only on explaining your reasoning.
- Be concise and precise.

